<a href="https://colab.research.google.com/github/fukutesta/Optimized/blob/main/Copy_of_ESRGAN_Manga_Upscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title [0] Giữ cho colab không tắt</font>


import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected");
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!");
            },60000);
    }

 function clearRefresher() {
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

In [ ]:
#@title [1] Kết nối với Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#!echo Phải chạy code lại nếu bị disconnect

Mounted at /content/drive


<font size=5><center>**ESRGAN**</center></font>



In [ ]:
#@title [2] Tải ESRGAN
#@markdown Notebook này sử dụng <a href="https://github.com/JoeyBallentine/ESRGAN/" target="_blank">ESRGAN fork of JoeyBallentine</a>
from IPython.display import clear_output
%cd /content
#!sudo apt-get install cuda
!git clone https://github.com/JoeyBallentine/ESRGAN.git
%cd /content/ESRGAN
!pip install -r requirements.txt
clear_output()
!echo Installed at /content/ESRGAN

Installed at /content/ESRGAN


In [ ]:
#@title [3] Tải và thiết lập các model
#@markdown <font size=2>Bạn có thể tải thêm model từ trang web này > <a href="https://upscale.wiki/wiki/Model_Database" target="_blank">https://upscale.wiki/wiki/Model_Database</a> dựa theo format các model bên dưới</font>
import ipywidgets as widgets
model_list = []
!pip install mega.py
from mega import Mega
mega = Mega()
m = mega.login()
%cd /content/ESRGAN/models

MangaJPEG = False #@param {type: "boolean"}
if MangaJPEG:
  m.download_url('https://mega.nz/file/MYEwyaSI#InO9qpK8NLJj6cVL2B3Z-CZehOc_qStGUUZNd-MJmNA')
  m.download_url('https://mega.nz/file/QBknBQgI#zeFPSLxoguPrLhAWPJHg4YvMT8BMa7nZuXGCzEpCs_w')
  m.download_url('https://mega.nz/file/9QUUSYYC#Bx8gmu2ZyNd-FemUfqbZOGcI9RRX2xc9orgvgP-KMyg')
  m.download_url('https://mega.nz/file/NYsSEbLT#NygiYv0F8OLGwMGA-gGyZhIA8g-V0__TbsIsXQ-TcQQ')
  model_list.extend(['1x_MangaJPEGHQPlus.pth','1x_MangaJPEGHQ.pth','1x_MangaJPEGMQ.pth','1x_MangaJPEGLQ.pth'])
#@markdown <font size=2>author: <i>Bunzero</i><br>scale: `1x`<br>models: `1x_MangaJPEGHQPlus.pth`, `1x_MangaJPEGHQ.pth`, `1x_MangaJPEGMQ.pth`, `1x_MangaJPEGLQ.pth`<br>info: <i>Remove noise and JPEG artefacts from images without disturing the screentones</i></font>

DigiManga = True #@param {type: "boolean"}
if DigiManga:
  m.download_url('https://mega.nz/file/m5syxKYB#SPsua_0RE-tWnM5s2GZzShHlwx1tMvRSLSOhuUhtVYc')
  model_list.append('4x_DigiManga.pth')
#@markdown <font size=2>author: <i>EULA</i><br>scale: `4x`<br>models: `4x_DigiManga.pth`<br>info: <i>Upscale B/W Manga Images to 4x with great screentone effect</i></font>

DigiMangav2 = False #@param {type: "boolean"}
if DigiManga:
  m.download_url('https://mega.nz/file/4ddQEYgB#ty2nUNX-ylp-UNKMLVSbjDVBAAGk9mwKCfyBshtnWpA')
  model_list.append('4x-eula-digimanga-bw-v2-nc1.pth')
#@markdown <font size=2>author: <i>EULA</i><br>scale: `4x`<br>models: `4x-eula-digimanga-bw-v2-nc1.pth`<br>info: <i>V1 Improvement</i></font>

LiloScale = False #@param {type: "boolean"}
if LiloScale:
  !gdown --id 1XcU9BALwsHnT922O9CA0_HqswlKs4244
  model_list.append('2xLiloScaleV0.2_80K.pth')
ReFocus = False #@param {type: "boolean"}
if ReFocus:
  m.download_url('https://mega.nz/file/hAVyxI6b#YzVeAfNerwKvUKZ8I6ESa-RhjW82NiU-O7g_QLLdIt8')
  model_list.append('1x-ReFocus-V3.pth')
#@markdown <font size=2>author: <i>Twittman</i><br>scale: `1x`<br>models: `1x-ReFocus-V3.pth`<br>info: <i>DeBlur, ReFocus, Sharpen Real life style images, but will work on Anime images too..</i></font>

original_RRDB_ESRGAN = False #@param {type: "boolean"}
if original_RRDB_ESRGAN:
  !gdown --id 1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene
  !gdown --id 1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN
  !gdown --id 1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-
  !gdown --id 1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr
  model_list.extend(['RRDB_ESRGAN_x4.pth','RRDB_PSNR_x4.pth','RRDB_ESRGAN_x4_old_arch.pth','RRDB_PSNR_x4_old_arch.pth'])
#@markdown <font size=2>author: <i>Xintao Wang and co.</i><br>scale: `4x`<br>models: `RRDB_ESRGAN_x4.pth`, `RRDB_PSNR_x4.pth`, `RRDB_ESRGAN_x4_old_arch.pth`, `RRDB_PSNR_x4_old_arch.pth`<br>info: <i>The original pre-trained models for upscaling</i></font>

transformed_ESRGAN = False #@param {type: "boolean"}
if transformed_ESRGAN:
  m.download_url('https://mega.nz/file/vtgSWKQT#K7Asn2zKe4N70R2aV89KEMTKhH3aiyGAAiuQDJF09qs')
  m.download_url('https://mega.nz/file/6kwQiCCS#v2uN8R44vVrlzmSqffGaCnzgogkPhhl67myJbuG45SA')
  model_list.extend(['2xESRGAN.pth','8xESRGAN.pth'])
#@markdown <font size=2>author: <i>victorca25</i><br>scale: `2x`, `8x`<br>models: `2xESRGAN.pth`, `8xESRGAN.pth`<br>info: <i>Transformed models from the original pre-trained model into the other scales.</i></font>





AnimeSharp4x = True #@param {type: "boolean"}
if AnimeSharp4x:
  m.download_url('https://mega.nz/file/CFZGHb4A#yo2NaVANd9-gk9n4lxA9zkzKPqVukZYpzendRVCinDw')
  model_list.append('4x-AnimeSharp.pth')
#@markdown <font size=2>author: <i>kim0</i><br>scale: `4x`<br>models: `4x-AnimeSharp.pth`<br>info: <i>Upscale colored mangas with a nice x4 upscaling</i></font>

AnimeSharp4x_lite = False #@param {type: "boolean"}
if AnimeSharp4x_lite:
  m.download_url('https://mega.nz/folder/bEoRQIRR#kEsaVHtwRL9vwfa5k2osyQ/file/zdwBhIZI')
  model_list.append('4x-AnimeSharp-lite.pth')
#@markdown <font size=2>author: <i>kim0</i><br>scale: `4x`<br>models: `4x-AnimeSharp-lite`<br>info: <i>Upscale colored mangas with a nice x4 upscaling (light version of 4x-AnimeSharp)</i></font>




MangaScale_V3 = False #@param {type: "boolean"}
if MangaScale_V3:
  m.download_url('https://mega.nz/file/nR51EYYC#RcAKkxb8hAzvZAj4TLxnAukd9gbrYuUeyjUp2l1ffb8')
  !gdown --id 1RGxHR-g-VjAgAr9XwBVjH1ydT-vlMXtQ
  model_list.append('2x_MangaScaleV3.pth')
#@markdown <font size=2>author: <i>Bunzero</i><br>scale: `2x`<br>model: `2x_MangaScaleV3.pth`<br>info: <i>upscaling q=70~ jpeg with implementing screentones</i></font>

Falcon_Fanart = False #@param {type: "boolean"}
if Falcon_Fanart:
  !gdown --id 1bqyG9llxkJ6i6MJTaNUiSgSXtrlPi0d7
  !7z e falcoon300.7z
  !rm ./falcoon300.7z
  model_list.append('falcoon300.pth')
#@markdown <font size=2>author: <i>LyonHrt</i><br>scale: `4x`<br>model: `falcoon300.pth`<br>info: <i>upscaling anime, manga digital art. Falcoon Fanart tries to improve upon Manga109Attempt's slightly blurriness with the goal of removing checkerboard patterns / and dithering. It has oil color based shading with sharp lines.</i></font>

DigiPaint = False #@param {type: "boolean"}
if DigiPaint:
  !gdown --id 103MX2bvd3GW0MYpC53ABU5VYY6v1t99Y
  model_list.append('DigiPaint35000.pth')
#@markdown <font size=2>author: <i>TheAtheistGod</i><br>scale: `4x`<br>model: `DigiPaint35000.pth`<br>info: <i>DigiPaint is a digital art upscaler designed to take brush strokes into account, based off of the Falcoon Fanart model.</i></font>

FatalimiX = False #@param {type: "boolean"}
if FatalimiX:
  !wget 'https://de-next.owncube.com/index.php/s/jYnFtncarkBmpcF/download?downloadFile' -O 4x_Fatality_Comix_260000_G.pth
  model_list.append('4x_Fatality_Comix_260000_G.pth')
#@markdown <font size=2>author: <i>Twittman</i><br>scale: `4x`<br>model: `4x_Fatality_Comix_260000_G.pth`<br>info: <i>FatalimiX is specialized in making low res comics to high res.</i></font>

Fatality_DeBlur = False #@param {type: "boolean"}
if Fatality_DeBlur:
  !wget 'https://de-next.owncube.com/index.php/s/aAojXwLTPZto8rP/download?downloadFile' -O 1x_Fatality_DeBlur_270000_G.pth
  model_list.append('1x_Fatality_DeBlur_270000_G.pth')
#@markdown <font size=2>author: <i>Twittman</i><br>scale: `1x`<br>model: `1x_Fatality_DeBlur_270000_G.pth`<br>info: <i>Fatality DeBlur is intended for general deblurring of images, and has some robustness against compression and noise.</i></font>

Baldrick_VHS_Fix_v0_2 = False #@param {type: "boolean"}
if Baldrick_VHS_Fix_v0_2:
  !wget 'https://icecube-eu-286.icedrive.io/download?p=Dsw65F5PttucA5GEjk5gTDoACdeM3oTN%2BrQDtV%2FiKP2VAtmmRiM0DiW8uIw4jbIXxER5myuy%2BuaWIRatD8bd0uJCntJjsTenc%2ByKYMPLWB83pOeTqxF%2FeVUS7OJLRfnJr7MOWInM002a%2BoZusB7Zy3LWg233LnrW703lny%2FG6d4s6dBVUaZtjLD1StShNKkOAtw3QzBhfBMU%2BN8NxNzmQz%2BobFFQm6heSvhEBqxRoBY%3D' -O 1xBaldrickVHSFix_180000_G_V0.2.pth
  model_list.append('1xBaldrickVHSFix_180000_G_V0.2.pth')
#@markdown <font size=2>author: <i>NimRodZorg</i><br>scale: `1x`<br>model: `1xBaldrickVHSFix_180000_G_V0.2.pth`<br>info: <i>Fixing minor VHS Chroma and Pattern Noise. May applicable for similar scanned images.</i></font>

jpg_BlueAmulet = False #@param {type: "boolean"}
if jpg_BlueAmulet:
  !gdown --id 1lgpo60okKC_9D6mzymHOvwdJDsOqxi7j
  !gdown --id 1V2PTYzMMDIB7NNCp-ZXhIf26a0xHO2b2
  !gdown --id 17ocFMDz4jfYH73mT_4i1099JCGVnOHd2
  model_list.extend(['1x_JPEG_40-60.pth','1x_JPEG_60-80.pth','1x_JPEG_80-100.pth'])
#@markdown <font size=2>author: <i>BlueAmulet</i><br>scale: `1x`<br>models: `1x_JPEG_40-60.pth`, `1x_JPEG_60-80.pth`, `1x_JPEG_80-100.pth`<br>info: <i>Denoising JPG compressed Images</i></font>

Jaywreck3 = False #@param {type: "boolean"}
if Jaywreck3:
  !wget 'https://icecube-eu-286.icedrive.io/download?p=Dsw65F5PttucA5GEjk5gTNHVbsvS0d9n4xG6iXebmHCnTeooDTK7XndlvQ%2FuyuS5wk43pZR81aUfwK0Qs%2FEg6B82OqzvZu1AgH79k%2FcnLQs6heqXMag1L1KDKpFQORZbjlHupFMKCpFpIZ%2BbnTm2GI%2F26D8nYJlgZ2aQDUDnql10yibDRmMxdS%2BwZggofl5YFr1qQ6KiOLY3RvYru4UnTW5J4jS8ksBNxa%2BQmbmw%2FQE%3D' -O 1x_NMKD-Jaywreck3-Lite_320k.pth
  !wget 'https://icecube-eu-286.icedrive.io/download?p=Dsw65F5PttucA5GEjk5gTNHVbsvS0d9n4xG6iXebmHCca1B9yE5n7QBqgoUooLeCEnt%2F55UtaHUigRVI1rEIbH2ijD5ZS7iFE89O%2FAkjhU5T4bTeZUGulQAPIoaKN85rPMCPVO%2FEMIoxB8SRx%2BgV5wyHpNRsRT6L7CS8yb7n99Ov0TtRBKIx9SgXyZPYlMXNvo1wW02WGBmFGtIJqSJsA1lLYjuTLMjIsAEuZSLrlvU%3D' -O 1x_NMKD-Jaywreck3-Soft-Lite_320k.pth
  model_list.extend(['1x_NMKD-Jaywreck3-Lite_320k.pth','1x_NMKD-Jaywreck3-Soft-Lite_320k.pth'])
#@markdown <font size=2>author: <i>nmkd</i><br>scale: `1x`<br>models: `1x_NMKD-Jaywreck3-Lite_320k.pth`, `1x_NMKD-Jaywreck3-Soft-Lite_320k.pth`<br>info: <i>Denoising JPG compressed Images. It is trained to handle any amount of JPEG compression, even removing artifacts of quality 5 JPEGs. Its only weakness is a slight tendency to oversharpen certain textures. It's recommended to not use this model for very small images</i></font>

SaiyaJin_DeJpeg = False #@param {type: "boolean"}
if SaiyaJin_DeJpeg:
  !wget 'https://f002.backblazeb2.com/file/ESRGAN/_RELEASE/1x_Saiyajin_DeJPEG_300000_G.pth'
  model_list.append('1x_Saiyajin_DeJPEG_300000_G.pth')
#@markdown <font size=2>author: <i>Twittman</i><br>scale: `1x`<br>model: `1x_Saiyajin_DeJPEG_300000_G.pth`<br>info: <i>Denoising JPG compressed Images from 2% to 100% quality.</i></font>

Colorizer = False #@param {type: "boolean"}
if Colorizer:
  !gdown --id 1UgxnymTfLaVhY3aaDuBRnTZ9ehckKVKJ
  model_list.append('1x_ColorizerV2_22000G.pth')
#@markdown <font size=2>author: <i>BlackScout</i><br>scale: `1x`<br>model: `1x_ColorizerV2_22000G.pth`<br>info: <i>Coloring black and white images.</i></font>
FSMangaV2 = False #@param {type: "boolean"}
if FSMangaV2:
  m.download_url('https://mega.nz/file/YMkRkbjC#rG4RI38RlHz5h12R94uD0ddhfLvfhNOYs2r39PmwjqY')
  model_list.append('4xFSMangaV2.pth')



clear_output()
model = widgets.Dropdown(options=model_list, value=model_list[0])
!echo 'Lựa chọn model'
model

Lựa chọn model


Dropdown(options=('4x_DigiManga.pth', '4x-eula-digimanga-bw-v2-nc1.pth', '4x-AnimeSharp.pth'), value='4x_DigiM…

In [ ]:
# import module
from PIL import Image
#@title [4] Chạy model
input_folder = '/content/drive/MyDrive/Nnn/input' #@param {type: "string"}
output_folder = '/content/drive/MyDrive/Nnn/output' #@param {type: "string"}
#@markdown <font size=2>if "output_folder" is empty, it'll create a "*_esrgan-ed" folder in the parent folder of "input_folder"</font>
import os
if output_folder.strip()=='':
  %cd '{input_folder}'
  ifolderpath = os.getcwd()
  ifolder = os.path.basename(ifolderpath)
  ofolder = ifolder + '_esrgan-ed'
  %cd ..
  %mkdir '{ofolder}'
  %cd '{ofolder}'
  output_folder = os.getcwd()

#@markdown ___
#@markdown Các config nâng cao, truy cập <a href="https://github.com/JoeyBallentine/ESRGAN/" target="_blank">https://github.com/JoeyBallentine/ESRGAN/</a> để biết thêm.
skip_existing_output_files = False #@param {type:"boolean"}
if skip_existing_output_files:
  skip_existing_output_files = ' --skip-existing'
else:
  skip_existing_output_files = ''
#@markdown <font size=1>if there are previously processed files in "output_folder", and you want to skip them instead of overwriting them.</font>
delete_input_files_after_process = False #@param {type:"boolean"}
if delete_input_files_after_process:
  delete_input_files_after_process = ' --delete-input'
else:
  delete_input_files_after_process = ''
seamless = 'tile' #@param ['tile','mirror','replicate','alpha_pad']
#@markdown <font size=1>Helps seamlessly process an image. tile = repeating along edges. mirror = reflected along edges. replicate = extended pixels along edges. alpha_pad = extended alpha border.</font>
use_CPU_instead_of_CUDA = False #@param {type:"boolean"}
if use_CPU_instead_of_CUDA:
  use_CPU_instead_of_CUDA = ' --cpu'
else:
  use_CPU_instead_of_CUDA = ''
#@markdown <font size=1>you can try using the TPU runtime type instead of GPU</font>
floating_point_16 = False #@param {type:"boolean"}
if floating_point_16:
  floating_point_16 = ' --floating-point-16'
else:
  floating_point_16 = ''
#@markdown <font size=1>FloatingPoint16/Halftensor type may increase process time in exchange for slight decrease on the accuracy.</font>
cache_max_split_depth = True #@param {type:"boolean"}
if cache_max_split_depth:
  cache_max_split_depth = ' --cache-max-split-depth'
else:
  cache_max_split_depth = ''
#@markdown <font size=1>If you're processing images of the same size, you can enable this to only calculate the automatic tile size once to improve performance.</font>
#@markdown ___
#@markdown <font size=2><center>**extra settings (alpha transparency)**</center></font>
binary_alpha = False #@param {type:"boolean"}
if binary_alpha:
  binary_alpha = ' --binary-alpha'
else:
  binary_alpha = ''
#@markdown <font size=1>Whether to use a 1 bit alpha transparency channel, Useful for PSX upscaling.</font>
ternary_alpha = False #@param {type:"boolean"}
if ternary_alpha:
  ternary_alpha = ' --ternary-alpha'
else:
  ternary_alpha = ''
#@markdown <font size=1>Whether to use a 2 bit alpha transparency channel, Useful for PSX upscaling.</font>
alpha_threshold =  0.5#@param {type:"number"}
#@markdown <font size=1>Only used when binary_alpha is supplied. Defines the alpha threshold for binary transparency.</font>
alpha_boundary_offset =  0.2#@param {type:"number"}
#@markdown <font size=1>Only used when binary_alpha is supplied. Determines the offset boundary from the alpha threshold for half transparency.</font>
alpha_mode = 'none' #@param ['none','bas','alpha_separately','swapping']
#@markdown <font size=1>Type of alpha processing to use. no_alpha = is no alpha processing. bas = is BA's difference method. alpha_separately = is upscaling the alpha channel separately (like IEU). swapping = is swapping an existing channel with the alpha channel.</font>
#@markdown ___
verbose = False #@param {type:"boolean"}
if verbose:
  verbose = ' --verbose'
else:
  verbose = ''
customize_model_usage = "" #@param {type:"string"}
if customize_model_usage.strip()=='':
  model_pth = model.value
else:
  model_pth = customize_model_usage

#@markdown <font size=1>Leave this empty if you use only one model that you have selected in the previous cell's output.<br>Else you can manually input the string for "model chaining" (eg- `1xSSAntiAlias9x.pth>4xBox.pth`) or "on-the-fly interpolation" (eg- `1x_model1.pth@50|1x_model2.pth@50>2x_model3.pth`).</font>

%cd /content/ESRGAN
cmd = f"python upscale.py --input '{input_folder}' --output '{output_folder}'{skip_existing_output_files}{delete_input_files_after_process} --seamless {seamless}{use_CPU_instead_of_CUDA}{floating_point_16}{cache_max_split_depth}{binary_alpha}{ternary_alpha} --alpha-threshold {alpha_threshold} --alpha-boundary-offset {alpha_boundary_offset} --alpha-mode {alpha_mode}{verbose} '{model_pth}'"
!echo $cmd
!$cmd
!echo ESRGAN process ended









/content/ESRGAN
python upscale.py --input /content/drive/MyDrive/Nnn/input --output /content/drive/MyDrive/Nnn/output --seamless tile --cache-max-split-depth --alpha-threshold 0.5 --alpha-boundary-offset 0.2 --alpha-mode none 4x-eula-digimanga-bw-v2-nc1.pth
Model: "models/4x-eula-digimanga-bw-v2-nc1.pth"
[17:27:14] WARNING  Truncating image channels                                         ]8;id=376811;file:///content/ESRGAN/upscale.py\upscale.py]8;;\:]8;id=549342;file:///content/ESRGAN/upscale.py#449\449]8;;\
Upscaling ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
ESRGAN process ended


In [ ]:
#@title [5] Resize ảnh
from PIL import Image
import os

# Set the input and output folders
input_folder = '/content/drive/MyDrive/Nnn/output'
output_folder = '/content/drive/MyDrive/Nnn/resize'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get the list of files in the input folder
file_list = os.listdir(input_folder)

# Loop through each file in the input folder
for file_name in file_list:
    # Construct the full path of the input file
    input_file_path = os.path.join(input_folder, file_name)

    # Open the image using Pillow
    image = Image.open(input_file_path)

    # Downscale the image by a factor of 4 using Lanczos interpolation
    width, height = image.size
    new_size = (width // 2.5, height //2.5)
    downscaled_image = image.resize(new_size, Image.LANCZOS)

    # Construct the full path of the output file
    output_file_path = os.path.join(output_folder, file_name)

    # Save the downscaled image to the output folder
    downscaled_image.save(output_file_path)

    print(f"Downscaled and saved: {output_file_path}")


NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/Nnn/output'

In [ ]:
#@title download the output_folder as 7zip
%cd '{output_folder}'
%cd ..
!7z a '{output_folder}' '{output_folder}/*'
clear_output()
import google.colab.files
out_7z = output_folder + '.7z'
downloaded = google.colab.files.download(out_7z)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>